In [ ]:
!pip install huggingface_hub


In [ ]:
from huggingface_hub import login
import re


In [ ]:
from google.colab import userdata
#hugging face hub is a platform for sharing, discovering, and using machine learning models
hf_token = userdata.get('HF-token')
login(token = hf_token)

In [ ]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report


/usr/local/lib/python3.11/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


In [ ]:
#read the csv file
df = pd.read_csv('/content/drive/MyDrive/AI/movie_reviews.csv')

In [ ]:
#take sample of 10
df = df.sample(n=10)

In [ ]:
df.head(10)

,review,sentiment
82,"Oh God, I must have seen this when I was only ...",negative
827,I thought before starting with these movie tha...,negative
469,Although the plot of this film is a bit far-fe...,positive
764,I first saw this film when I was about 8 years...,positive
404,"Gamers: DR is not a fancy made movie, it's mor...",positive
653,A group of friends break down in the middle of...,positive
465,One of the more intelligent serial killer movi...,positive
802,This is a great show with total freshness and ...,positive
457,The Man With a Golden Arm was one of a trio of...,positive
376,Doctor Mordrid is one of those rare films that...,positive


In [ ]:
#convert values negative and positive to 0 and 1 respectively
df['sentiment'] = df['sentiment'].apply(lambda x: 0 if x == 'negative' else 1)

In [ ]:
df.head(10)

,review,sentiment
82,"Oh God, I must have seen this when I was only ...",0
827,I thought before starting with these movie tha...,0
469,Although the plot of this film is a bit far-fe...,1
764,I first saw this film when I was about 8 years...,1
404,"Gamers: DR is not a fancy made movie, it's mor...",1
653,A group of friends break down in the middle of...,1
465,One of the more intelligent serial killer movi...,1
802,This is a great show with total freshness and ...,1
457,The Man With a Golden Arm was one of a trio of...,1
376,Doctor Mordrid is one of those rare films that...,1


In [ ]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"
model = AutoModelForSequenceClassification.from_pretrained(model_name) #loads a pre-trained model from Hugging Face’s model hub
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token #sets the padding token (pad_token) to be the same as the end-of-sequence token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.1-8B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
#Agent 1 prompt will give some sort of direct sentiment like if it is positive or negative
from torch.nn import Softmax

class SentimentAgent:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def predict_sentiment(self, review_text):
        prompt = f"Please classify the sentiment of this movie review as either positive or negative: {review_text}"
        inputs = self.tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)
        outputs = self.model(**inputs) #forwards the tokenized text through the model.
        sentiment = outputs.logits.argmax(dim=-1).item()
        return sentiment


In [ ]:
#Agent 2 prompt will give output as negative or positive on the basis of acting, direction, and story
class AspectAgent:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def predict_aspect_sentiment(self, review_text):
        prompt = f"Please classify the sentiment of the following review based on the acting, direction, and story: {review_text}"
        inputs = self.tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)
        outputs = self.model(**inputs) #Forwards the tokenized text through the model.
        sentiment = outputs.logits.argmax(dim=-1).item() #argmax(dim=-1): selects the index with the highest logit value.
        return sentiment

In [ ]:
#Agent 3 prompt will just try to determine intensity of the review which will help further to apply the majoriy voting
class IntensityAgent:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def predict_sentiment(self, review_text):
        prompt = f"Please determine the intensity of sentiment in this review. Is it strong or weak?  {review_text}"
        inputs = self.tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)
        outputs = self.model(**inputs)
        sentiment = outputs.logits.argmax(dim=-1).item()
        return sentiment

In [ ]:
#head of sentiment
print(df['sentiment'].head())

82     0
827    0
469    1
764    1
404    1
Name: sentiment, dtype: int64


In [ ]:
from collections import Counter
#Counter is a dictionary subclass that helps count hashable objects

def aggregate_predictions(sentiment_pred, aspect_pred, intensity_pred): #aggregates the three sentiment predictions using majority voting.

    # Collect all predictions in a list
    predictions = [sentiment_pred, aspect_pred, intensity_pred]

    # Perform majority voting
    vote_count = Counter(predictions)  # Count occurrences of each label
    final_sentiment = vote_count.most_common(1)[0][0]  # Get the most common label

    return {
        'final_sentiment': final_sentiment
    }



In [ ]:
# Initialize agents with the Llama 3.1 8B model
sentiment_agent = SentimentAgent(model, tokenizer)
aspect_agent = AspectAgent(model, tokenizer)
intensity_agent = IntensityAgent(model, tokenizer)

In [ ]:
predictions = []
for index, row in df.iterrows():
    review_text = row['review']
    true_label = row['sentiment']  # Assuming 'sentiment_label' is the true sentiment

    # Get predictions from all three agents
    sentiment_pred = sentiment_agent.predict_sentiment(review_text)
    aspect_pred = aspect_agent.predict_aspect_sentiment(review_text)
    intensity_pred = intensity_agent.predict_sentiment(review_text)

    #plot some output after one iteration
    print(sentiment_pred, aspect_pred, intensity_pred)
    # Aggregate the results
    final_prediction = aggregate_predictions(sentiment_pred, aspect_pred, intensity_pred)
    predictions.append(final_prediction)

1 1 1
0 1 0
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1


In [ ]:
print(df['sentiment'].head(10))

82     0
827    0
469    1
764    1
404    1
653    1
465    1
802    1
457    1
376    1
Name: sentiment, dtype: int64


In [ ]:
predictions_df = pd.DataFrame(predictions)

# Now you can use head() on the DataFrame
predictions_df.head(10)

,final_sentiment
0,1
1,0
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


In [ ]:
#calculate accuracy and classification report

final_sentiments = [pred['final_sentiment'] for pred in predictions]

print("Accuracy:", accuracy_score(df['sentiment'], final_sentiments))
print("Classification Report:")
print(classification_report(df['sentiment'], final_sentiments))

Accuracy: 0.9
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       0.89      1.00      0.94         8

    accuracy                           0.90        10
   macro avg       0.94      0.75      0.80        10
weighted avg       0.91      0.90      0.89        10



The recall for the negative class is less as compared to the positive class this can be because the data sample consists of more positive labels and less negative. This eventually leads to less f1-score as it is harmonic mean of precision and recall also called as trade-off.